In [2]:
from client_conf import ALICE_DOCKER, BOB_DOCKER, CHARLIE_DOCKER, BITCOIN_DOCKER
from lnd import LndClient
from client import BtcClient
import logging as logger
import schedule

ModuleNotFoundError: No module named 'schedule'

Bitcoin enables a new kind of payment, called a "streaming payment".
You can send fraction of satoshis to a peer in a periodic time range

## Connect to Bitcoin Core and Lightning Deamon


In [4]:
alice = LndClient(ALICE_DOCKER)
bob =  LndClient(BOB_DOCKER)
charlie =  LndClient(BOB_DOCKER)
bitcoin_node = BtcClient(BITCOIN_DOCKER)

print("Alice PubKey: ", alice.pubkey)
print("Bob PubKey: ", bob.pubkey)

INFO: CONNECTING TO Alice: 10.21.21.9:10009
INFO: CONNECTING TO Bob: 10.21.21.20:10009


Alice PubKey:  02ac1de718a6f13abf333761e8cb005cb771886b3802b1b40ceaff904cb965a472
Bob PubKey:  02f5b1d78ba8db8cb6d0e4f616e6976b27292a28b6aca34cf640e5284fec0e7f4c


## Fund and open channel

In [5]:
def fund_node(bitcoin_node, ln_node, amount):
    address = ln_node.address().address
    logger.info(f"Funding {ln_node} with {amount} BTC at {address}")
    bitcoin_node.sendtoaddress(address, amount)
    bitcoin_node.generate(6)
    logger.info(ln_node.wallet_balance())
    logger.info("Funding complete")  

In [6]:
fund_node(bitcoin_node, alice, 4)
fund_node(bitcoin_node, bob, 3)

INFO: Funding Alice with 4 BTC at bcrt1q2t7xvh04cx76j8g38qnlthm9w8xctc7n86mz2p
DEBUG: Starting new HTTP connection (1): 10.21.21.8:18443
DEBUG: http://10.21.21.8:18443 "POST / HTTP/1.1" 200 98
DEBUG: Starting new HTTP connection (1): 10.21.21.8:18443
DEBUG: http://10.21.21.8:18443 "POST / HTTP/1.1" 200 78
DEBUG: Starting new HTTP connection (1): 10.21.21.8:18443
DEBUG: http://10.21.21.8:18443 "POST / HTTP/1.1" 200 435
INFO: {'node': 'Alice', 'total_balance': 400000000, 'confirmed_balance': 400000000, 'unconfirmed_balance': 0}
INFO: Funding complete
INFO: Funding Bob with 3 BTC at bcrt1q8286u6ywe80uzsausdhdtguy3u0nzqyxcahm5m
DEBUG: Starting new HTTP connection (1): 10.21.21.8:18443
DEBUG: http://10.21.21.8:18443 "POST / HTTP/1.1" 200 98
DEBUG: Starting new HTTP connection (1): 10.21.21.8:18443
DEBUG: http://10.21.21.8:18443 "POST / HTTP/1.1" 200 78
DEBUG: Starting new HTTP connection (1): 10.21.21.8:18443
DEBUG: http://10.21.21.8:18443 "POST / HTTP/1.1" 200 435
INFO: {'node': 'Bob', 'to

In [9]:
def open_channel(node, pubkey, amount):
    
    logger.info(f"[{node}] Opening channel with {pubkey}..")
    if not node.channel_exists_with_node(pubkey):    
        params = {
            "node_pubkey_string": pubkey,
            "local_funding_amount": amount
        }
        node.open_channel(**params)
        logger.info(f"[{node}] Channel with {pubkey} opened!")
    else: 
        logger.info(f"[{node}] Channel with {pubkey} already exists")    

In [10]:
open_channel(alice, bob.pubkey, 1000000)

INFO: [Alice] Opening channel with 02f5b1d78ba8db8cb6d0e4f616e6976b27292a28b6aca34cf640e5284fec0e7f4c..
ERROR: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "peer 02f5b1d78ba8db8cb6d0e4f616e6976b27292a28b6aca34cf640e5284fec0e7f4c is not online"
	debug_error_string = "{"created":"@1654127596.396432382","description":"Error received from peer ipv4:10.21.21.9:10009","file":"src/core/lib/surface/call.cc","file_line":952,"grpc_message":"peer 02f5b1d78ba8db8cb6d0e4f616e6976b27292a28b6aca34cf640e5284fec0e7f4c is not online","grpc_status":2}"
>
Traceback (most recent call last):
  File "/home/pyclient/code/lnd.py", line 307, in open_channel
    response = self.client.OpenChannelSync(request)
  File "/usr/local/lib/python3.10/site-packages/grpc/_channel.py", line 946, in __call__
    return _end_unary_response_blocking(state, call, False, None)
  File "/usr/local/lib/python3.10/site-packages/grpc/_channel.py", line 849, in _end_unary_response_blocking
 

## Create a stream of payments

In [ ]:
total_sats = 50000 # Total number of satoshi 
periodic_sats = 1000 # Number of sats to send every interval
tot_interval = total_sats // periodic_sats


def periodic_payment(node, pubkey, amount, intervals):
    """
    Schedule a payment to be sent to the target node
    """
    if not node.channel_exists_with_node(pubkey):
        open_channel(node, pubkey, amount)
        
    node.schedule_payment(pubkey, amount, interval)
    logger.info(f"[{node}] Payment scheduled!")
    


In [ ]:
# schedule.every().thursday.at("17:00").do(periodic_payment)
schedule.every(3).seconds.do(periodic_payment)
